In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io
import json

with open(r"keys.json", "r") as f:
    data = json.load(f)

connection_string = data.get("connection_string_azure")
container_name = "containerforregular"
csv_blob_name = "HPC_NEW/final_df/vectorized_df_v1.parquet"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

def download_blob_to_dataframe(blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    download_stream = blob_client.download_blob()
    data = download_stream.readall()
    file_like_object = io.BytesIO(data)
    df = pd.read_parquet(file_like_object)
    del data, file_like_object
    return df

df = download_blob_to_dataframe(csv_blob_name)
print("Parquet DataFrame:")
print(df.head(3))

import gc
gc.collect()

Parquet DataFrame:
                                      Vector_Content  Label
0  [-0.5991044, 0.14731808, 0.32865947, -0.244333...      1
1  [0.10335503, -0.24981976, 0.14990778, 0.367329...      1
2  [-0.75803685, 0.8188284, 0.6138587, 0.532273, ...      1


17

In [ ]:
import numpy as np

X = np.array(df["Vector_Content"].tolist(), dtype=np.float32)
y = df['Label']

print(X.dtype)

float32


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rfc = RandomForestClassifier(n_estimators=400, max_depth=30, min_samples_leaf=1, min_samples_split=2)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred_rfc))

              precision    recall  f1-score   support

           0       0.87      0.77      0.82     72262
           1       0.80      0.88      0.84     72764

    accuracy                           0.83    145026
   macro avg       0.83      0.83      0.83    145026
weighted avg       0.83      0.83      0.83    145026

